In [1]:
import cv2
from PIL import Image
from PIL import ImageTk
import threading
import tkinter as tk
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from scipy.spatial import distance
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os
import glob

def detect_and_predict_mask(frame, faceNet, maskNet):
        
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
            (104.0, 177.0, 123.0))

     
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    faces = []
    locs  = []
    preds = []
    
        
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))

      
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)
        
    return (locs, preds)

def button1_clicked(videoloop_stop):
    threading.Thread(target=videoLoop, args=(videoloop_stop,)).start()


def button2_clicked(videoloop_stop):
    videoloop_stop[0] = True

prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
maskNet = load_model("mask_new.model")
MIN_DIST = 400
dist =0
color1=0


print("[INFO] starting video stream...")
def videoLoop(mirror=False):
    No = 0
    cap = cv2.VideoCapture(No)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 800)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 600)
    face_model = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    while True:
        ret,frame = cap.read()
        frame = imutils.resize(frame, width=1000)
        faces = face_model.detectMultiScale(frame,scaleFactor=1.1, minNeighbors=4)
        label1 = "Safe Distance Maintaind"

        (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

        for (box, pred) in zip(locs, preds):
            (startX, startY, endX, endY) = box
            (mask, withoutMask,incmask) = pred


            if mask > withoutMask and mask > incmask:
                label = "Mask" 
                color = (0, 255, 0)
            elif incmask > mask and incmask > withoutMask:
                label = "Inccorect Mask" 
                color = (255, 0, 0)
            else:
                label = "No Mask" 
                color = (0, 0, 255)

            label = "{}: {:.2f}%".format(label, max(mask, withoutMask,incmask) * 100)

            cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
            i=0;
            for j in range(i+1, len(faces)):
                dist = distance.euclidean(faces[i][:3],faces[j][:3])

                if dist<MIN_DIST:
                    label1 = "Social Distance not maintaind"
                    color1 =(0, 255, 0)
                    cv2.putText(frame, label1, (startX-50, startY - 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)       
                    cv2.rectangle(frame, (startX-50, startY-30), (endX+50, endY+50), (0, 0, 255), 2)

        

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageTk.PhotoImage(image)
        panel = tk.Label(image=image)
        panel.image = image
        panel.place(x=50, y=70)
        

        if videoloop_stop[0]:

            videoloop_stop[0] = False
            panel.image = 0
            image1 = Image.open("C:/Users/DELL/Desktop/Markshit/Face-Mask-Detection-master/bb.jpg")
            test = ImageTk.PhotoImage(image1)

            label1 = tk.Label(image=test)
            label1.image = test
            label1.place(x=100, y=70)
            break



videoloop_stop = [False]

root = tk.Tk()
root.geometry("1920x1080+0+0")
root.title("Face Mask Detection System")
button1 = tk.Button(
    root, text="start", bg="#fff", font=("", 50),
    command=lambda: button1_clicked(videoloop_stop))
button1.place(x=1100, y=100, width=400, height=200)

button2 = tk.Button(
    root, text="stop", bg="#fff", font=("", 50),
    command=lambda: button2_clicked(videoloop_stop))
button2.place(x=1100, y=360, width=400, height=200)

root.mainloop()

[INFO] starting video stream...
